In [4]:
import sys
sys.path.append("E:\\Document\\bitcoin\\trade-executor")

In [16]:
from typing import List
from IPython.display import display
import pandas as pd
import numpy as np
from tradingstrategy.chain import ChainId
from tradingstrategy.exchange import Exchange

universe = client.fetch_exchange_universe()

# Have nice type decoration to make the tutorial
# more readable
exchanges: List[Exchange] = []
volumes: List[float] = []
chains: List[str] = []

for xchg in universe.get_top_exchanges_by_30d_volume()[0:10]:
    exchanges.append(xchg.name)
    # Format volume with the  thousand separator
    volumes.append("{:,.2f}".format(xchg.vol_30d))
    # Pull blockchain name for the chain id number from
    # chain data database
    chain_id: ChainId = xchg.chain_id
    chains.append(chain_id.get_name())

# Convert output to Pandas DataFrame object so that
# notebook can render it as a nice table
df = pd.DataFrame({"Exchange": exchanges, "Blockchain": chains, "USD Volume 30d": volumes})

# Index rows starting with one instead of zero
df.index = np.arange(1, len(df)+1)

# Show dataframe as HTML table
display(df)

,Exchange,Blockchain,USD Volume 30d
1,Uniswap v3,Ethereum,"15,700,288,962.16"
2,PancakeSwap v2,Binance Smart Chain,"3,937,321,754.48"
3,Uniswap v2,Ethereum,"2,387,449,057.93"
4,Sushi,Ethereum,"812,686,176.65"
5,Biswap,Binance Smart Chain,"352,203,688.80"
6,Quickswap,Polygon,"296,028,913.35"
7,Shiba Swap,Ethereum,"199,760,357.19"
8,Trader Joe,Avalanche C-chain,"151,839,062.19"
9,FSTSwap,Binance Smart Chain,"107,906,860.92"
10,GIBX Swap,Binance Smart Chain,"98,211,185.12"


In [49]:
from tradingstrategy.exchange import ExchangeUniverse
columnar_pair_table = client.fetch_pair_universe()
all_pairs_dataframe = columnar_pair_table.to_pandas()

In [63]:
import pandas as pd
from tradingstrategy.chain import ChainId

# Convert PyArrow table to Pandas format to continue working on it
all_pairs_dataframe = columnar_pair_table.to_pandas()

# Filter down to pairs that only trade on Sushiswap
uniswap_swap = universe.get_by_chain_and_slug(ChainId.ethereum, "pancakeswap-v2")
uniswap_pairs: pd.DataFrame = all_pairs_dataframe.loc[all_pairs_dataframe['exchange_id'] == uniswap_swap.exchange_id]

print(f"Uniswap V3 on Ethereum has {len(uniswap_pairs)} pairs")

Uniswap V3 on Ethereum has 90738 pairs


In [64]:
uniswap_pairs = uniswap_pairs.copy()
total_volume_30d = uniswap_pairs["buy_volume_30d"] + uniswap_pairs["sell_volume_30d"]
uniswap_pairs["total_30d_volume"] = total_volume_30d
# Sort by highest volume firs   t
uniswap_pairs = uniswap_pairs.sort_values("total_30d_volume", ascending=False)

print("total_30d_volume column sum added")

total_30d_volume column sum added


In [179]:
output = uniswap_pairs[["base_token_symbol", "quote_token_symbol", "total_30d_volume"]].copy()

output.style.format({
  'total_30d_volume': lambda val: f'${val:,.2f}',
})

output = output.rename(columns={
    'base_token_symbol': 'Base token',
    'quote_token_symbol': 'Quote token',
    'total_30d_volume': 'Volume 30d USD'})

top_10 = output.head(20)

top_10.style.format({
  'Volume 30d USD': lambda val: f'${val:,.2f}',
}).hide_index()

C:\Users\DESKTOP\AppData\Local\Temp\ipykernel_2204\715529809.py:16: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  }).hide_index()


Base token,Quote token,Volume 30d USD
WBNB,USDT,"$526,214,208.00"
WBNB,BUSD,"$493,658,976.00"
BUSD,USDT,"$175,676,544.00"
BTCB,BUSD,"$66,874,752.00"
YES,USDT,"$64,042,272.00"
MEER,USDT,"$55,941,312.00"
TIME,USDT,"$55,936,336.00"
BUSD,USDC,"$51,551,632.00"
Cake,BUSD,"$49,301,684.00"
Cake,WBNB,"$47,391,216.00"


## Pair Trading

In [117]:
import datetime
import pandas as pd

from tradingstrategy.chain import ChainId
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.strategy.strategy_module import StrategyType, TradeRouting, ReserveCurrency

# Tell what trade execution engine version this strategy needs to use
trading_strategy_engine_version = "0.1"

# What kind of strategy we are running.
# This tells we are going to use
trading_strategy_type = StrategyType.managed_positions

# How our trades are routed.
# PancakeSwap basic routing supports two way trades with BUSD
# and three way trades with BUSD-BNB hop.
trade_routing = TradeRouting.pancakeswap_basic

# How often the strategy performs the decide_trades cycle.
# We do it for every 16h.
trading_strategy_cycle = CycleDuration.cycle_16h

# Strategy keeps its cash in BUSD
reserve_currency = ReserveCurrency.busd

# Time bucket for our candles
candle_time_bucket = TimeBucket.h1

# Which chain we are trading
chain_id = ChainId.bsc

# Which exchange we are trading on.
exchange_slug = "pancakeswap-v2"

# Which trading pair we are trading
trading_pair_tickers = [("WBNB", "BUSD"), ("BTCB", "BUSD")]

# How much of the cash to put on a single trade
position_size = 0.10

#
# Strategy thinking specific parameter
#

batch_size = 90

slow_ema_candle_count = 15

fast_ema_candle_count = 5


# Range of backtesting and synthetic data generation.
# Because we are using synthetic data actual dates do not really matter -
# only the duration

start_at = datetime.datetime(2021, 6, 1)

end_at = datetime.datetime(2022, 1, 1)

# Start with 10,000 USD
initial_deposit = 10_000


In [125]:
import pandas_ta
help(pandas_ta.bbands)

Help on function bbands in module pandas_ta.volatility.bbands:

bbands(close, length=None, std=None, ddof=0, mamode=None, talib=None, offset=None, **kwargs)
    Bollinger Bands (BBANDS)
    
    A popular volatility indicator by John Bollinger.
    
    Sources:
        https://www.tradingview.com/wiki/Bollinger_Bands_(BB)
    
    Calculation:
        Default Inputs:
            length=5, std=2, mamode="sma", ddof=0
        EMA = Exponential Moving Average
        SMA = Simple Moving Average
        STDEV = Standard Deviation
        stdev = STDEV(close, length, ddof)
        if "ema":
            MID = EMA(close, length)
        else:
            MID = SMA(close, length)
    
        LOWER = MID - std * stdev
        UPPER = MID + std * stdev
    
        BANDWIDTH = 100 * (UPPER - LOWER) / MID
        PERCENT = (close - LOWER) / (UPPER - LOWER)
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): The short period. Default: 5
        std (int): The long 

In [172]:
from fileinput import close
from typing import List, Dict

from pandas_ta.overlap import ema

from tradeexecutor.state.visualisation import PlotKind
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pricing_model import PricingModel
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager
from tradeexecutor.state.state import State
from tradingstrategy.universe import Universe

from sklearn.linear_model import LinearRegression

def decide_trades(
        timestamp: pd.Timestamp,
        universe: Universe,
        state: State,
        pricing_model: PricingModel,
        cycle_debug_data: Dict) -> List[TradeExecution]:
    """The brain function to decide the trades on each trading strategy cycle.

    - Reads incoming execution state (positions, past trades)

    - Reads the current universe (candles)

    - Decides what to do next

    - Outputs strategy thinking for visualisation and debug messages

    :param timestamp:
        The Pandas timestamp object for this cycle. Matches
        trading_strategy_cycle division.
        Always truncated to the zero seconds and minutes, never a real-time clock.

    :param universe:
        Trading universe that was constructed earlier.

    :param state:
        The current trade execution state.
        Contains current open positions and all previously executed trades, plus output
        for statistics, visualisation and diangnostics of the strategy.

    :param pricing_model:
        Pricing model can tell the buy/sell price of the particular asset at a particular moment.

    :param cycle_debug_data:
        Python dictionary for various debug variables you can read or set, specific to this trade cycle.
        This data is discarded at the end of the trade cycle.

    :return:
        List of trade instructions in the form of :py:class:`TradeExecution` instances.
        The trades can be generated using `position_manager` but strategy could also hand craft its trades.
    """

    # The pair we are trading
    ids = universe.pairs.get_all_pair_ids()
    pair1 = universe.pairs.get_pair_by_id(ids[0])
    pair2 = universe.pairs.get_pair_by_id(ids[1])


    # How much cash we have in the hand
    cash = state.portfolio.get_current_cash()

    # Get OHLCV candles for our trading pair as Pandas Dataframe.
    # We could have candles for multiple trading pairs in a different strategy,
    # but this strategy only operates on single pair candle.
    # We also limit our sample size to N latest candles to speed up calculations.

    candles: pd.DataFrame = universe.candles.get_all_samples_by_range(timestamp-pd.Timedelta(1,'d'),timestamp)

    candles1 = candles[candles['pair_id']==pair1.pair_id]
    candles2 = candles[candles['pair_id']==pair2.pair_id]
    # We have data for open, high, close, etc.
    # We only operate using candle close values in this strategy.
    close1 = candles1["close"]
    close2 = candles2["close"]
    # regression
    lr = LinearRegression().fit(close1.values.reshape(-1,1), close2.values.reshape(-1,1))
    residual = close2.values.reshape(-1,1) - lr.predict(close1.values.reshape(-1,1))
    residual = pd.Series(residual.reshape(-1), index=close1.index)
    
    residual = (residual-residual.mean())/residual.std()

    # Get the last close price from close time series
    # that's Pandas's Series object
    # https://pandas.pydata.org/docs/reference/api/pandas.Series.iat.html
    # current_price = close1.iloc[-1]

    # List of any trades we decide on this cycle.
    # Because the strategy is simple, there can be
    # only zero (do nothing) or 1 (open or close) trades
    # decides
    trades = []

    # Create a position manager helper class that allows us easily to create
    # opening/closing trades for different positions
    position_manager = PositionManager(timestamp, universe, state, pricing_model)

    if residual[-1] >= 3:
        # Entry condition:
        # Residual higher than 3
        if not position_manager.is_any_open():
            buy_amount = cash * position_size
            trades += position_manager.open_1x_long(pair2, buy_amount)
    elif residual[-1] <= -3:
        # Entry condition:
        # Residual lower than -3
        if not position_manager.is_any_open():
            buy_amount = cash * position_size
            trades += position_manager.open_1x_long(pair1, buy_amount)
    elif residual[-1]<1 and residual[-1]>-1:
        # Exit condition:
        if position_manager.is_any_open():
            trades += position_manager.close_all()
    
    # Visualize strategy
    # See available Plotly colours here
    # https://community.plotly.com/t/plotly-colours-list/11730/3?u=miohtama
    visualisation = state.visualisation
    visualisation.plot_indicator(timestamp, "Residual", PlotKind.technical_indicator_on_price, residual, colour="darkblue")

    return trades

In [173]:
from typing import Optional
from tradeexecutor.strategy.trading_strategy_universe import load_all_data, TradingStrategyUniverse, \
    load_pair_data_for_single_exchange
from tradeexecutor.strategy.execution_context import ExecutionContext
from tradingstrategy.client import Client
import datetime

def create_trading_universe(
        ts: datetime.datetime,
        client: Client,
        execution_context: ExecutionContext,
        candle_time_frame_override: Optional[TimeBucket]=None,
) -> TradingStrategyUniverse:
    """Creates the trading universe where the strategy trades.

    If `execution_context.live_trading` is true then this function is called for
    every execution cycle. If we are backtesting, then this function is
    called only once at the start of backtesting and the `decide_trades`
    need to deal with new and deprecated trading pairs.

    As we are only trading a single pair, load data for the single pair only.

    :param ts:
        The timestamp of the trading cycle. For live trading,
        `create_trading_universe` is called on every cycle.
        For backtesting, it is only called at the start

    :param client:
        Trading Strategy Python client instance.

    :param execution_context:
        Information how the strategy is executed. E.g.
        if we are live trading or not.

    :param candle_timeframe_override:
        Allow the backtest framework override what candle size is used to backtest the strategy
        without editing the strategy Python source code file.

    :return:
        This function must return :py:class:`TradingStrategyUniverse` instance
        filled with the data for exchanges, pairs and candles needed to decide trades.
        The trading universe also contains information about the reserve asset,
        usually stablecoin, we use for the strategy.
    """

    # Load all datas we can get for our candle time bucket
    dataset = load_pair_data_for_single_exchange(
        client,
        execution_context,
        candle_time_bucket,
        chain_id,
        exchange_slug,
        trading_pair_tickers,
        )

    
    # Filter down to the single pair we are interested in
    universe = TradingStrategyUniverse.create_limited_pair_universe(
        dataset,
        chain_id,
        exchange_slug,
        trading_pair_tickers,
        ("BTCB", "BUSD")
    )

    return universe

In [174]:
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in C:\Users\DESKTOP/.tradingstrategy


In [175]:
import logging

from tradeexecutor.backtest.backtest_runner import run_backtest_inline

state, universe, debug_dump = run_backtest_inline(
    name="BNB/USD EMA crossover example",
    start_at=start_at,
    end_at=end_at,
    client=client,
    cycle_duration=trading_strategy_cycle,
    decide_trades=decide_trades,
    create_trading_universe=create_trading_universe,
    initial_deposit=initial_deposit,
    reserve_currency=ReserveCurrency.busd,
    trade_routing=TradeRouting.pancakeswap_basic,
    log_level=logging.WARNING,
)

trade_count = len(list(state.portfolio.get_all_trades()))
print(f"Backtesting completed, backtested strategy made {trade_count} trades")

Backtesting BNB/USD EMA crossover example, 2021-06-01-2022-01-01 at 2021-12-31, total 8 trades: : 18547200it [00:30, 604964.57it/s]                            

Backtesting completed, backtested strategy made 8 trades


In [176]:
from tradeexecutor.analysis.trade_analyser import build_trade_analysis

analysis = build_trade_analysis(state.portfolio)

In [177]:
from IPython.core.display_functions import display

summary = analysis.calculate_summary_statistics()

with pd.option_context("display.max_row", None):
    display(summary.to_dataframe())

,0
Trading period length,190 days
Return %,2.84%
Annualised return %,5.43%
Cash at start,"$10,000.00"
Value at end,"$10,283.58"
Trade win percent,75.00%
Total trades done,4
Won trades,3
Lost trades,1
Stop losses triggered,0


In [178]:
from tradeexecutor.analysis.trade_analyser import expand_timeline

timeline = analysis.create_timeline()

expanded_timeline, apply_styles = expand_timeline(
        universe.universe.exchanges,
        universe.universe.pairs,
        timeline)

# Do not truncate the row output
with pd.option_context("display.max_row", None):
    display(apply_styles(expanded_timeline))

Remarks,Opened at,Duration,Exchange,Base asset,Quote asset,Position max size,PnL USD,PnL %,Open price USD,Close price USD,Trade count
,2021-06-01,1 days 8 hours,PancakeSwap v2,WBNB,BUSD,"$2,187.56",$187.56,18.76%,$344.372013,$408.961057,2
,2021-07-15,16 hours,PancakeSwap v2,WBNB,BUSD,"$2,032.82",$-4.69,-0.46%,$315.796570,$314.342564,2
,2021-10-04,1 days 8 hours,PancakeSwap v2,BTCB,BUSD,"$2,090.80",$54.23,5.33%,"$48,785.615834","$51,383.677650",2
,2021-12-08,16 hours,PancakeSwap v2,WBNB,BUSD,"$2,093.91",$46.49,4.54%,$588.204134,$614.915343,2


In [144]:
from tradeexecutor.visual.single_pair import visualise_single_pair

figure = visualise_single_pair(
    state,
    universe.universe.candles,
    start_at=start_at,
    end_at=end_at)

figure.show()

AssertionError: visualise_single_pair() can be only used for a trading universe with a single pair